In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

plt.rcParams['font.family'] = "AppleGothic"

import json
with open('/Users/hyo-cheolahn/Documents/Projects/Book-Recommendation/data/scraping/all_book_data_ver3[0:20000].json', "r") as file:
    data = json.load(file)

In [2]:
import pandas as pd

texts = []
for el in data:
    t1 = [x.split(':')[-1].strip() for x in el['description'].split('\n')] # 여러개의 라인으로 된 데이터 정리
    texts.append(' '.join(t1))
df = pd.DataFrame()
df['texts'] = texts
print(len(df))
df.head()



8907


,texts
0,행복을 찾는 방법이 아니라불행에 대한 수비력을 길러주는58가지 인생 이야기《1cm ...
1,"좋은 감각을 지니려면,디자인을 잘하려면,더 나은 브랜드를 만들려면 어떻게 해야 하는..."
2,“그렇게 많은 문상훈을 봤는데도여전히 새로운 문상훈의 얼굴이 이 책에 있다.” (작...
3,“코난밖에 모르던 내가 일본어로 먹고살아요‘그저 좋아하다가 잘하게 된 센님의 일본어...
4,"마지막을 알기에 더 충만해지는 삶,죽음의 관점에서 새롭게 해석되는살아 있는 오늘의 ..."


In [3]:
# import pandas as pd


# texts_ks = []
# for el in data:
#     t1 = [x.split(':')[-1].strip() for x in el['key_sentences'].split('\n')] # 여러개의 라인으로 된 데이터 정리
#     texts_ks.append(' '.join(t1))

# df['texts_ks'] = pd.Series(texts_ks)
# df.head()
# len(df)


In [4]:

# series_combined = df['texts'].str.cat(df['texts_ks'], sep=' | ')
# df_combined = series_combined.to_frame(name="texts")

# df_combined.head()

In [5]:
df_combined = df

In [6]:
from bertopic import BERTopic
print("BERTopic imported successfully!")

/opt/anaconda3/envs/mecab_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BERTopic imported successfully!


In [7]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab
from tqdm import tqdm

In [8]:
import MeCab

# 사전 경로를 '-d' 옵션으로 전달
mecab = MeCab.Tagger('-d /opt/homebrew/lib/mecab/dic/mecab-ko-dic')
print(mecab.parse("한국어 형태소 분석 테스트 중입니다."))

한국어	NNG,*,F,한국어,Compound,*,*,한국/NNG/*+어/NNG/*
형태소	NNG,*,F,형태소,Compound,*,*,형태/NNG/*+소/NNG/*
분석	NNG,행위,T,분석,*,*,*,*
테스트	NNG,행위,F,테스트,*,*,*,*
중	NNB,*,T,중,*,*,*,*
입니다	VCP+EF,*,F,입니다,Inflect,VCP,EF,이/VCP/*+ᄇ니다/EF/*
.	SF,*,*,*,*,*,*,*
EOS



In [9]:
df_combined
if df_combined.columns.size == 1:
    documents = df_combined.iloc[:, 0].tolist()
len(documents)

preprocessed_documents = []

for line in tqdm(documents):
  # 빈 문자열이거나 숫자로만 이루어진 줄은 제외
  if line and not line.replace(' ', '').isdecimal():
    preprocessed_documents.append(line)

100%|██████████| 8907/8907 [00:00<00:00, 205898.67it/s]


In [10]:

class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [11]:
# from konlpy.tag import Mecab

# # 사전 경로를 지정하여 Mecab 초기화
# mecab = Mecab(dicpath='/opt/homebrew/lib/mecab/dic/mecab-ko-dic')

# # 테스트
# print(mecab.morphs("한국어 형태소 분석 테스트입니다."))

In [12]:
import MeCab
mecab = MeCab.Tagger('-d /opt/homebrew/lib/mecab/dic/mecab-ko-dic')
print(mecab.parse("테스트입니다."))

테스트	NNG,행위,F,테스트,*,*,*,*
입니다	VCP+EF,*,F,입니다,Inflect,VCP,EF,이/VCP/*+ᄇ니다/EF/*
.	SF,*,*,*,*,*,*,*
EOS



In [13]:
from konlpy.tag import Mecab

# 사전 경로를 명시하여 Mecab 객체 생성
mecab = Mecab(dicpath='/opt/homebrew/lib/mecab/dic/mecab-ko-dic')

# CustomTokenizer에 Mecab 객체 전달
custom_tokenizer = CustomTokenizer(mecab)


In [14]:
tokens = custom_tokenizer("한국어 형태소 분석 테스트입니다.")
print(tokens)  # 결과가 ['한국어', '형태소', '분석', '테스트', '입니다'] 같은 리스트인지 확인

['한국어', '형태소', '분석', '테스트', '입니다']


In [15]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)



In [16]:
print(type(preprocessed_documents))# preprocessed_documents 확인
print(preprocessed_documents[:5])  # 샘플 데이터 출력

# 올바른 형식 예시
# ["문장 하나입니다.", "또 다른 문장입니다."]

<class 'list'>
['행복을 찾는 방법이 아니라불행에 대한 수비력을 길러주는58가지 인생 이야기《1cm 다이빙》, 《홈 in 홈》으로 많은 사랑을 받았던 에세이스트 태수가 2년 만의 신작 《어른의 행복은 조용하다》로 돌아왔다. 이번 신작에서 저자 태수는 그동안 선보였던 이야기보다 한층 성숙하고 현명하게 삶의 행복에 가까워지는 방법에 대해 이야기한다. 새로운 것, 짜릿한 것, 남들보다 높은 곳에서 행복하고 싶어 발버둥치는 사람들에게 행복은 꼭 그런 데에만 있는 게 아니라 불행해지지 않는 것에서부터 출발한다고 조용히 일러준다.《어른의 행복은 조용하다》는 요란한 세상에서도 흔들리지 않고 내 삶을 살아가는 튼튼하고 단단한 태도를 담아냈다. 시끌벅적 기쁜 일을 찾아다니기보다도, 울 일이 없고 별다른 나쁜 일이 없는 하루를 만들어야 한다고 강조한다. 그랬을 때에야 비로소 진정한 행복이 우리 곁에 온다는 사실을 다시 한번 일깨워준다.이 책을 먼저 읽은 독자들의 반응 역시 뜨겁다. “울고 싶어지는 날이면 태수 작가의 글을 찾는다. 충분히 울고 다시 나아가기 위해”, “『불편한 편의점』 이후로 오랜만에 끝나지 않길 바라던 책”이라며 극찬하하며, 저자의 SNS에서 5만여 명의 독자에게 선보인 선공개 원고에도 빨리 책으로 출간해달라는 요청이 쇄도했다.이제 당신 차례다. 《어른의 행복은 조용하다》를 읽고 현명하게 행복을 찾아보자. 혹자는 여전히 의심스러운 목소리로 그토록 조용한 인생에서도 행복을 발견할 수 있냐고 묻지만 저자는 확신에 찬 목소리로 단호하게 답한다. “물론”이라고.', '좋은 감각을 지니려면,디자인을 잘하려면,더 나은 브랜드를 만들려면 어떻게 해야 하는가‘일’하는 사람의 섬세한 ‘감각’ 탐구조수용의 첫 단독 에세이한 호에 하나씩, 균형 잡힌 브랜드를 선정하여 그 철학과 감성, 이야기를 소개하는 매거진 『B』의 발행인 조수용의 에세이 『일의 감각』이 출간되었다. 『일의 감각』은 조수용의 첫 단독 저서로, 처음 일을 시작했을 때부터 지금까지 어떤 마음으로 일해왔

In [17]:
# train_bow_embeddings = vectorizer.fit_transform(preprocessed_documents)# train_bow_embeddings를 NumPy 배열로 변환
train_bow_embeddings = vectorizer.fit_transform(preprocessed_documents).toarray()
print(train_bow_embeddings)


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 3]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [18]:
print(train_bow_embeddings.shape)


(8906, 3000)


In [19]:
vocab = vectorizer.get_feature_names()
id2token = {k: v for k, v in zip(range(0, len(vocab)), vocab)}


In [21]:
len(vocab)

3000

In [23]:
train_contextualized_embeddings = bert_embeddings_from_list(
    preprocessed_documents,
    "sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens",
    max_seq_length=512,  # 모델이 처리할 최대 토큰 길이 설정
    batch_size=32        # 원하는 배치 크기 설정
)

Batches: 100%|██████████| 279/279 [02:50<00:00,  1.63it/s]


In [24]:
qt = TopicModelDataPreparation()

training_dataset = qt.load(train_contextualized_embeddings, train_bow_embeddings, id2token)
     

In [31]:
import os

# tokenizers 병렬 처리 비활성화
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [25]:
ctm = CombinedTM(bow_size=len(vocab), contextual_size=768, n_components=50, num_epochs=20)
ctm.fit(training_dataset)

0it [00:00, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingfac

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Epoch: [1/20]	 Seen Samples: [8896/178120]	Train Loss: 787.414514335797	Time: 0:01:39.179180: : 1it [01:39, 99.18s/it]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if 

In [26]:
ctm.get_topics(5)

defaultdict(list,
            {0: ['아이', '엄마', '하늘', '행복', '부족'],
             1: ['으로', '에서', '한다', '자신', '사람'],
             2: ['인생', '생각', '저자', '지만', '자신'],
             3: ['에서', '으로', '에게', '사람', '사랑'],
             4: ['작가', '으로', '에서', '문학', '소설'],
             5: ['인생', '자신', '저자', '시작', '아니'],
             6: ['사람', '이야기', '저자', '자신', '인생'],
             7: ['사랑', '그림', '순간', '일상', '처럼'],
             8: ['자연', '사진', '식물', '일상', '그림'],
             9: ['따뜻', '일상', '아무튼', '고양이', '웃음'],
             10: ['모여', '버스', '정면', '의심', '합리'],
             11: ['작가', '으로', '시인', '에서', '산문'],
             12: ['다움', '합리', 'sbs', '개념', '강요'],
             13: ['낯설', '합리', '챕터', '이라고', '텃밭'],
             14: ['에서', '이야기', '작가', '함께', '사람'],
             15: ['서울', '도시', '역사', '전쟁', '최초'],
             16: ['아이', '함께', '저자', '교사', '교육'],
             17: ['가사', '복원', '이게', '위하', '말기'],
             18: ['문학', '작품', '산문', '시인', '문학상'],
             19: ['에게', '행복', '마음', '당신', '위로'],
     

In [28]:
ctm.get_topic_lists()

[['아이', '엄마', '하늘', '행복', '부족', '마을', '에게', '카메라', '청년', '부모'],
 ['으로', '에서', '한다', '자신', '사람', '저자', '이야기', '우리', '에게', '라는'],
 ['인생', '생각', '저자', '지만', '자신', '한다', '아니', '환자', '라고', '경험'],
 ['에서', '으로', '에게', '사람', '사랑', '작가', '우리', '이야기', '시간', '마음'],
 ['작가', '으로', '에서', '문학', '소설', '시인', '라는', '산문', '까지', '독자'],
 ['인생', '자신', '저자', '시작', '아니', '환자', '한다', '생각', '어떤', '위해'],
 ['사람', '이야기', '저자', '자신', '인생', '에서', '대한', '으로', '한다', '세상'],
 ['사랑', '그림', '순간', '일상', '처럼', '마음', '작가', '따뜻', '사진', '당신'],
 ['자연', '사진', '식물', '일상', '그림', '나무', '고양이', '풍경', '여행', '에세이'],
 ['따뜻', '일상', '아무튼', '고양이', '웃음', '다가', '세이', '그림', '기쁨', '매일'],
 ['모여', '버스', '정면', '의심', '합리', '평생', '눈부신', '참여', '써야', '낯설'],
 ['작가', '으로', '시인', '에서', '산문', '우리', '한다', '독자', '라는', '에게'],
 ['다움', '합리', 'sbs', '개념', '강요', '모양', '낯설', '의심', '정면', '관하'],
 ['낯설', '합리', '챕터', '이라고', '텃밭', '갇혀', '얼굴', '되찾', '의심', '재밌'],
 ['에서', '이야기', '작가', '함께', '사람', '일상', '사랑', '여행', '으로', '시간'],
 ['서울', '도시', '역사', '전쟁', '최초', '국내', '공개',

In [32]:
import pyLDAvis as vis

lda_vis_data = ctm.get_ldavis_data_format(vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)

100%|██████████| 140/140 [01:31<00:00,  1.54it/s]
